In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
files = [
    "UCLA1-1130444-all-leads.csv",
    "UCLA2-4442242-all-leads.csv",
    "UCLA3-5728091-all-leads.csv"
]

# Load and print first 10 lines of each file
for file in files:
    print(f"\n--- First 10 lines of {file} ---\n")
    try:
        df = pd.read_csv(file)
        print(df.head(10))  # Show first 10 rows
    except Exception as e:
        print(f"Failed to read {file}: {e}")


--- First 10 lines of UCLA1-1130444-all-leads.csv ---

                         Email Email Status Lead First Name Lead Last Name  \
0                          NaN          NaN           Nirav          Patel   
1  jeroen.vantilburg@ionity.eu    CATCH_ALL          Jeroen    Van Tilburg   
2                          NaN          NaN           Brian         Lillie   
3                          NaN          NaN            Lara          Rabiu   
4                          NaN          NaN         Liliana      C  Rieger   
5              krhee@nachc.org        VALID             Kyu           Rhee   
6                          NaN          NaN         Allison          Stone   
7                          NaN          NaN  José Alexander         Herran   
8                          NaN          NaN            Noel    Castellanos   
9                          NaN          NaN           Ruchi         Bondre   

          Lead Full Name                            Lead Job Title  \
0            Ni

  Email Email Status Lead First Name Lead Last Name     Lead Full Name  \
0   NaN          NaN            Seth          Koshy         Seth Koshy   
1   NaN          NaN            Jack         Nilles               Jack   
2   NaN          NaN        Jeremy A          Lyons    Jeremy A  Lyons   
3   NaN          NaN          Steven        Diebold     Steven Diebold   
4   NaN          NaN            Cali        Gilbert       Cali Gilbert   
5   NaN          NaN           Suraj      Palaparty    Suraj Palaparty   
6   NaN          NaN           Yusuf          Yucel        Yusuf Yucel   
7   NaN          NaN         Rydhima      Shah Brar  Rydhima Shah Brar   
8   NaN          NaN             Eti        El-Kiss        Eti El-Kiss   
9   NaN          NaN           Gabby         Stuart       Gabby Stuart   

                                  Lead Job Title  \
0                   Founder/CEO/District Manager   
1                                            CEO   
2  Co-Founder | Recruiting Op

In [3]:
def read_csv_simple(path):
    return pd.read_csv(path, encoding="utf-8-sig")

In [4]:
def preprocess_min(df, source_name):
    df = df.copy()
    df["__source_file"] = source_name

    if "Email" in df.columns:
        df["Email"] = (df["Email"].astype(str)
                       .str.strip().str.lower()
                       .replace({"": pd.NA, "nan": pd.NA}))
    if "Email Status" in df.columns:
        df["Email Status"] = (df["Email Status"].astype(str)
                              .str.strip().str.upper()
                              .replace({"": pd.NA, "NAN": pd.NA}))
    if "Company Domain" in df.columns:
        df["Company Domain"] = (df["Company Domain"].astype(str)
                                .str.strip().str.lower()
                                .replace({"": pd.NA, "nan": pd.NA}))
    if "Lead Linkedin Url" in df.columns:
        df["Lead Linkedin Url"] = (df["Lead Linkedin Url"].astype(str)
                                   .str.strip()
                                   .replace({"": pd.NA, "nan": pd.NA}))
    return df

In [5]:
# 1) Read + minimal normalization
dfs = [preprocess_min(read_csv_simple(p), os.path.basename(p)) for p in files]

# 2) Schema union
all_cols = sorted(set().union(*[set(d.columns) for d in dfs]))
aligned = []
for df in dfs:
    for c in all_cols:
        if c not in df.columns:
            df[c] = pd.NA
    aligned.append(df[all_cols])

stacked = pd.concat(aligned, ignore_index=True)

In [6]:
# 3) De-duplicate
has_email = stacked["Email"].notna() if "Email" in stacked.columns else pd.Series(False, index=stacked.index)
df_with_email = stacked[has_email].copy()
df_no_email  = stacked[~has_email].copy()

status_score = {"VALID": 2, "CATCH_ALL": 1}
df_with_email["__status_score"] = df_with_email.get("Email Status").map(status_score).fillna(0).astype(int)

# completeness on user columns only (exclude helper cols)
helper_cols = {"__status_score"}
value_cols = [c for c in df_with_email.columns if c not in helper_cols]
df_with_email["__completeness"] = df_with_email[value_cols].notna().sum(axis=1)

source_order = {os.path.basename(p): i for i, p in enumerate(files)}
df_with_email["__source_priority"] = df_with_email["__source_file"].map(source_order).fillna(len(files)).astype(int)

# sort preference
df_with_email = df_with_email.sort_values(
    by=["__status_score", "__completeness", "__source_priority"],
    ascending=[False, False, True]
)

# Group-wise consolidation + change tracking
def consolidate_group(g):
    # g already sorted with best row first
    best = g.iloc[0].copy()
    base_before = best.copy()  # snapshot to detect changes after consolidation
    for col in g.columns:
        if col.startswith("__"):
            continue
        if pd.isna(best[col]):
            for v in g[col].iloc[1:]:
                if pd.notna(v):
                    best[col] = v
                    break
    # Which columns changed due to consolidation?
    changed_cols = []
    for col in g.columns:
        if col.startswith("__"):
            continue
        old = base_before[col]
        new = best[col]
        # consider NaN equality
        if (pd.isna(old) and pd.notna(new)) or (pd.notna(old) and pd.isna(new)) or (pd.notna(old) and pd.notna(new) and old != new):
            changed_cols.append(col)
    # Prepare a small record for change_log
    kept = best.copy()
    kept["__changed_columns"] = ", ".join(changed_cols) if changed_cols else ""
    kept["__n_rows_in_group"] = len(g)
    kept["__group_sources"] = ", ".join(g["__source_file"].astype(str).unique().tolist())
    kept["__group_statuses"] = ", ".join(g["Email Status"].astype(str).fillna("NA").unique().tolist())
    return kept

consolidated = df_with_email.copy()

# For rows without Email: drop exact duplicates
no_email_clean = df_no_email.drop_duplicates()

merged = pd.concat([consolidated, no_email_clean], ignore_index=True)

In [8]:
# 4) Build change logs / summary
n_before = len(stacked)
n_after  = len(merged)

# Count emails that appear >1 time in the final data (kept, not collapsed)
if "Email" in merged.columns:
    email_counts_final = merged["Email"].value_counts(dropna=True)
    emails_repeated_final = email_counts_final[email_counts_final > 1]
    n_emails_repeated_final = int((emails_repeated_final > 1).sum())
else:
    n_emails_repeated_final = 0

print("=== Summary ===")
print(f"Rows before: {n_before}")
print(f"Rows after:  {n_after}")
print(f"Emails that appear >1 time (kept): {n_emails_repeated_final}")


=== Summary ===
Rows before: 4358
Rows after:  4358
Emails that appear >1 time (kept): 1


In [9]:
# Summary table
summary_rows = [{
    "files_merged": ", ".join(os.path.basename(p) for p in files),
    "rows_before": n_before,
    "rows_after": n_after,
    "duplicates_removed_total": n_before - n_after,
    "email_groups_with_duplicates": n_email_groups_gt1,
    "rows_without_email_before": n_no_email_before,
    "rows_without_email_after": n_no_email_after,
    "no_email_exact_dups_removed": n_no_email_dups_removed,
}]
summary_df = pd.DataFrame(summary_rows)

In [10]:
print(summary_df)

                                        files_merged  rows_before  rows_after  \
0  UCLA1-1130444-all-leads.csv, UCLA2-4442242-all...         4358        4358   

   duplicates_removed_total  email_groups_with_duplicates  \
0                         0                             1   

   rows_without_email_before  rows_without_email_after  \
0                       3206                      3206   

   no_email_exact_dups_removed  
0                            0  


In [13]:
if "Email" in merged.columns:
    dup_counts = merged["Email"].value_counts(dropna=True)
    dup_emails = dup_counts[dup_counts > 1]
    print("Duplicate emails kept:\n", dup_emails.to_string())

    # Show the actual rows for those emails
    dup_rows = merged[merged["Email"].isin(dup_emails.index)].sort_values(["Email"])
    print("\nFirst few duplicate rows:\n", dup_rows.head(20))

Duplicate emails kept:
 Email
eric@eonetwork.org    2

First few duplicate rows:
      Company Domain  Company Followers  Company Founded In  \
588   eonetwork.org           127728.0              1987.0   
1065  eonetwork.org              637.0                 NaN   

                        Company Industry Company Li Company Type  \
588             Non-profit Organizations               Nonprofit   
1065  Professional Training and Coaching              Non Profit   

                                 Company Li Description  \
588   The Entrepreneurs’ Organization (EO) is a high...   
1065  At EO Accelerator, being one of EO's primary g...   

                                       Company Linkedin Company Location City  \
588   https://www.linkedin.com/company/entrepreneursorg            Alexandria   
1065  https://www.linkedin.com/company/eo-accelerato...                Dubai    

     Company Location Country Code Company Location Country Name  ...  \
588                            

In [14]:
#Descending Order by Company Founded In YEAR
if "Company Founded In" in merged.columns:
    merged = merged.sort_values(by="Company Founded In", ascending=False)

#Download
out_csv = "UCLA-merged-all-leads.csv"
merged.to_csv(out_csv, index=False)

print(f"Merged CSV saved to: {out_csv}  (rows={len(merged)})")

Merged CSV saved to: UCLA-merged-all-leads.csv  (rows=4358)


In [15]:
dfOut = pd.read_csv("UCLA-merged-all-leads.csv")
print(dfOut.head(10))

                     Company Domain  Company Followers  Company Founded In  \
0                               NaN                1.0              2025.0   
1                               NaN                1.0              2025.0   
2                    motormeter.org                3.0              2025.0   
3                  ammapartners.com                9.0              2025.0   
4                    csuite-law.com                1.0              2025.0   
5                               NaN                9.0              2025.0   
6            aspirelegalnetwork.com                1.0              2025.0   
7                       futuxre.com                1.0              2025.0   
8  myprivateprofessorfoundation.org               17.0              2025.0   
9                    blairheard.com                3.0              2025.0   

             Company Industry Company Li Company Type  \
0  Retail Apparel and Fashion     Sole Proprietorship   
1                 Real Esta

In [16]:
# What columns do we have?
print(sorted(merged.columns))

# Look for any plausible name columns (case-insensitive)
print([c for c in merged.columns if "name" in c.lower()])


['Company Domain', 'Company Followers', 'Company Founded In', 'Company Industry', 'Company Li Company Type', 'Company Li Description', 'Company Linkedin', 'Company Location City', 'Company Location Country Code', 'Company Location Country Name', 'Company Location Timezone', 'Company Location Timezone Offset', 'Company Name', 'Company Size', 'Company Specialities', 'Company Website', 'Email', 'Email Status', 'Lead Connection Degree', 'Lead First Name', 'Lead Full Name', 'Lead Job Title', 'Lead Last Name', 'Lead Li Picture Url', 'Lead Linkedin Url', 'Lead Location City', 'Lead Location Country Name', 'Lead Location Raw', 'Lead Location State', 'Lead Months In Company', 'Lead Months In Position', 'Lead Open Profile', 'Lead Position Started Month', 'Lead Position Started Year', 'Lead Premium', 'Lead Summary', 'Lead Years In Company', 'Lead Years In Position', '__completeness', '__source_file', '__source_priority', '__status_score']
['Company Location Country Name', 'Company Name', 'Lead Fi